In [5]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle

In [2]:
# Get processed data
df = pd.read_csv('preprocessed_data 2.csv', header=0)


,rating,title,text,language,language_label,label,clean_text_1,clean_text_2,clean_text_4,clean_text_5,final_text
0,1,mens ultrasheer,"This model may be ok for sedentary types, but ...",en,English,0,"This model may be ok for sedentary types, but ...","This model may be ok for sedentary types , but...",this model may be ok for sedentary types but ...,model may ok sedentary type active get alot jo...,model may ok sedentary type active get alot jo...
1,4,Surprisingly delightful,This is a fast read filled with unexpected hum...,en,English,2,This is a fast read filled with unexpected hum...,This is a fast read filled with unexpected hum...,this is fast read filled with unexpected humo...,fast read filled unexpected humour profound in...,fast read filled unexpected humour profound in...
2,2,"Works, but not as advertised",I bought one of these chargers..the instructio...,en,English,0,I bought one of these chargers..the instructio...,I bought one of these chargers .. the instruct...,bought one of these chargers the instruction...,bought one charger instruction say light stay ...,bought one charger instruction say light stay ...
3,2,Oh dear,I was excited to find a book ostensibly about ...,en,English,0,I was excited to find a book ostensibly about ...,I was excited to find a book ostensibly about ...,was excited to find book ostensibly about fe...,excited find book ostensibly feminism volume n...,excited find book ostensibly feminism volume n...
4,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod...",en,English,0,"I am a big JVC fan, but I do not like this mod...","I am a big fan , but I do not like this model ...",am big fan but do not like this model wa...,big fan not model suspiscious saw several unit...,big fan not model suspiscious saw several unit...
...,...,...,...,...,...,...,...,...,...,...,...
249379,3,3 stars plus.,"Spellbound was Paula Abduls peak, Although its...",en,English,1,"Spellbound was Paula Abduls peak, Although its...","was Abduls peak , Although its her nd # album ...",was abduls peak although its her nd album s...,abduls peak nd album selling million copy thin...,abduls peak nd album selling million copy thin...
249380,3,It's ok,"This is my second purchase of this album used,...",en,English,1,"This is my second purchase of this album used,...","This is my purchase of this album used , lmao ...",this is my purchase of this album used lmao ...,purchase album used lmao music style changed m...,purchase album used lmao music style changed m...
249381,2,Not Great But Not Awful,This product would be a good buy for under $10...,en,English,0,This product would be a good buy for under $. ...,This product would be a good buy for under $ ....,this product would be good buy for under th...,product would good buy completed robot go shor...,product would good buy completed robot go shor...
249382,5,Greatest movie ever,I love the movie the five heart beats I think ...,en,English,2,I love the movie the five heart beats I think ...,I love the movie the heart beats I think it wa...,love the movie the heart beats think it was ...,love movie heart beat think one gratest movie ...,love movie heart beat think one gratest movie ...


## Aspect-Based Sentiment Analysis

In [8]:
# pickle.dump(clf_nb,open("model_naive_bayes.pkl",'wb'))

# Get pickle file for TF-IDF
tfidf = pickle.load(open("tfidf.pkl",'rb'))

# Get pickle file for model
model = pickle.load(open("model_naive_bayes.pkl",'rb'))

In [59]:
# Sample data

# sample_text = 4000

# clean_text_2 - this is the 2nd cleaned text in the DF
clean_text_2 = df['clean_text_2'][sample_text]

# final_text - this is the final cleaned text after doing all preprocessing steps
final_text = df['final_text'][sample_text]

# this is an input from the user. keywords that they want to analyze.
aspect_list = ['lyrics']

print(clean_text_2)

Cause and Effect is not that great of a band , but they seem to be a strong influence on Faith Assembly . Think droning male lead vocals , with synth lines straight from the bedroom studio . This would not be so bad in itself , but some of the lyrics either swim through so much cliche and/or are such cheesy groaners that I personally can not keep myself from poking for the track advance button on my player . If you are stuck in a circa   is synth pop rut and do not have terribly discerning tastes , this album is for you . For the rest of you , may I suggest something along the lines of ; is The Sunrise E.P. instead for contemporary synth pop of a similiarly mellow style . Cheers !


In [60]:
# Class names
class_names = ['Negative', 'Positive']

# Function to get the sentiment analysis for a specific text
def SentimentAnalysis(inText):
    
    # Create a dataframe containing the input text
    df_actual = pd.DataFrame([inText], columns=['final_text'])
    
    # Vectorization using TF-IDF
    x_actual_matrix = tfidf.transform(df_actual['final_text'])
    x_actual_tfidf_vector = x_actual_matrix.toarray()

    # Predict the class: Negative/Positive
    y_actual_pred = model.predict(x_actual_tfidf_vector)
    
    # Class probabilities
    class_prob = model.predict_proba(x_actual_tfidf_vector)
    
    sentiment = class_names[y_actual_pred[0]]
    
    # Return the sentiment and class probabilities
    return sentiment, class_prob

In [61]:
# Function to get the sentiment analysis for a specific aspect/keyword
def AspectBasedSentimentAnalysis(inText):
    
    # Sentence tokenization
    sent_list = sent_tokenize(str(inText))
    
    # Initialize dictionaries
    aspect_class = {}
    aspect_prob_diff = {}
    
    # Loop through all aspects/keywords submitted by user
    for aspect in aspect_list:
        
        # For every aspect, loop through all sentences
        for sentence in sent_list:
            
            # If aspect is in the sentence
            if aspect in sentence:
                
                # Remove the aspect from the sentence
                aspect_text =  sentence.replace(aspect, "")
                
                # Perform sentiment analysis on the sentence without the keyword
                aspect_sentiment = SentimentAnalysis(aspect_text)
                
                # Get the difference between negative and positive class probabilities
                # If the class probabilities are so close to each other (e.g., negative=0.49, positive=0.51), the difference is not too clear
                prob_diff = abs(aspect_sentiment[1][0][0] - aspect_sentiment[1][0][1])
                
                # print(aspect_text,aspect_sentiment[0],aspect_sentiment[1][0])
                
                # This will output only if the probability difference at least 10 PPS
                if (prob_diff >= 0.10) and (aspect_prob_diff.get(aspect) is not None and prob_diff > aspect_prob_diff.get(aspect) ) or ( (prob_diff >= 0.10) and (aspect_prob_diff.get(aspect) is None) ):
                    
                    # Update class dictionary
                    aspect_class[aspect] = aspect_sentiment[0]
                    
                    # Update class probability dictionary
                    aspect_prob_diff[aspect] = prob_diff
                                        
    # Return sentiment analysis for the aspect
    return aspect_class                    

In [63]:
# Get overall sentiment
overall_sentiment = SentimentAnalysis(final_text)

# Perform aspect-based sentiment analysis
aspect_sentiments = AspectBasedSentimentAnalysis(clean_text_2)

# Display results
print("Overall Sentiment:", overall_sentiment[0])

# Print results
print("\nAspect Sentiments:")
for aspect, sentiment in aspect_sentiments.items():
    print(f"  {aspect}: {sentiment}")

Overall Sentiment: Negative

Aspect Sentiments:
  lyrics: Negative
